In [1]:
import pandas as pd
import requests
import json
import datadotworld as dw
%load_ext watermark
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

ModuleNotFoundError: No module named 'toc'

In [173]:
%watermark -a 'Lukas Hager' -v -p pandas,datadotworld

Author: Lukas Hager

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

pandas      : 1.1.3
datadotworld: 1.7.0



#### Define the API query string and pull the data

In [3]:
qs = 'https://api.census.gov/data/2019/acs/acs5?key=dbdcf56ed39f351aba8093a27cb1e19f1f3ac043&get=B19013_001E&for='

income_df = pd.DataFrame(data = json.loads(requests.get('{}zip%20code%20tabulation%20area:*'.format(qs)).text))
# rename
income_df.rename(columns={0: 'med_income', 1: 'state', 2: 'zip_code'}, inplace=True)
# drop the headers
income_df.drop(income_df.index[0], inplace=True)
# filter to TX
income_df = income_df.loc[income_df.state == '48']
# cast median income as integer and zip as string
income_df['med_income'] = income_df['med_income'].astype('float')
income_df['zip_code'] = income_df['zip_code'].astype('str')


In [175]:
income_df.loc[income_df.med_income >= 0].head()

,med_income,state,zip_code
25859,72512.0,48,75001
25860,101429.0,48,75002
25861,64462.0,48,75006
25862,86648.0,48,75007
25863,111146.0,48,75009


#### What does the distribution look like?
There are fewer zip codes than google would suggest, but this seems overall somewhat reasonable, at least in terms of magnitude

In [176]:
income_df.loc[income_df.med_income >= 0]['med_income'].describe()

count      1781.00
mean      60038.24
std       24911.51
min        2499.00
25%       44006.00
50%       54438.00
75%       69464.00
max      250001.00
Name: med_income, dtype: float64

#### Not sure how to gut check this, but the median income across TX matches google

In [177]:
tx_med_income = json.loads(requests.get('{}state:48'.format(qs)).text)[1][0]
print('The median household income in Texas is {}'.format(tx_med_income))


The median household income in Texas is 61874


#### Get the raw data from data.world

In [178]:
files = dw.load_dataset('tji/officer-involved-shootings').dataframes
shot_officers = files['shot_officers']
shot_civilians = files['shot_civilians']

/Users/hlukas/opt/anaconda3/lib/python3.8/site-packages/datadotworld/models/dataset.py:206: UserWarning: Unable to set data frame dtypes automatically using shot_civilians schema. Data types may need to be adjusted manually. Error: Bool column has NA values in column 138
  warnings.warn(


#### Perform the join

In [179]:
shot_officers['incident_zip'] = shot_officers['incident_zip'].astype('str')
shot_officers_income = shot_officers.set_index('incident_zip').join(income_df.set_index('zip_code'), how = 'left')
shot_civilians['incident_zip'] = shot_civilians['incident_zip'].astype('str')
shot_civilians['incident_zip'] = shot_civilians['incident_zip'].str.replace(pat = '(\\.\d)', repl = '')
shot_civilians_income = shot_civilians.set_index('incident_zip').join(income_df.set_index('zip_code'), how = 'left')

#### Merge is good for officers, not so good for civilians

In [180]:
print('{} null income(s) in officers data'.format(sum(shot_officers_income['med_income'].isnull())))
print('{} null income(s) in civlian data'.format(sum(shot_civilians_income['med_income'].isnull())))

1 null income(s) in officers data
11 null income(s) in civlian data


#### No census data on the one officer data issue, but the zip code should be 78040 according to Google Maps

In [181]:
shot_officers_income.loc[shot_officers_income['med_income'].isnull() == True]

,date_ag_received,date_incident,incident_address,incident_city,incident_county,officer_harm,officer_name_first,officer_name_last,officer_age,officer_race,...,civilian_name_first_3,civilian_name_last_3,civilian_age_3,civilian_race_3,civilian_gender_3,media_link_1,media_link_2,media_link_3,med_income,state
78042,2018-10-12,2018-09-15,417 E. FREMONT STREET,LAREDO,WEBB,INJURY,NaN,NaN,29.0,HISPANIC,...,NaN,NaN,NaN,NaN,NaN,LMT ONLINE,KGNS TV,NaN,NaN,NaN


In [182]:
income_df.loc[income_df['zip_code'] == '78042']

,med_income,state,zip_code


#### We see the same issue with civilians -- otherwise good

In [183]:
shot_civilians_income.loc[shot_civilians_income['med_income'].isnull() == True]

,date_ag_received,date_incident,incident_address,incident_city,incident_county,incident_result_of,incident_call_other,civilian_name_first,civilian_name_last,civilian_name_full,...,num_rows_about_this_incident,officer_age_11,officer_gender_11,officer_on_duty,officer_race_11,time_incident,weapon_reported_by_media,weapon_reported_by_media_category,med_income,state
75261,NaN,2015-09-08,1500 RANGE ROAD,"DFW, AIRPORT",TARRANT,OTHER - SPECIFY TYPE OF CALL,TRAINING EXERCISE - RICOCHET FRAGMENTS RESULTI...,NaN,NaN,NaN,...,1,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
77637,2020-11-12,2020-11-10,700 TEXAS AVE.,NEDERLAND,JEFFERSON,OTHER,ASSIST OTHER AGENCY IN SEARCHING FOR AN ARMED ...,DARON,JONES,DARON JONES,...,1,NaN,NaN,True,NaN,22:18:00,GUN,FIREARM,NaN,NaN
78042,2018-10-12,2018-09-15,417 E. FREMONT STREET,LAREDO,WEBB,TRAFFIC STOP; OTHER,DOMESTIC VIOLENCE CALL/EVADING ARREST,ANTONIO,AGUIRRE,ANTONIO AGUIRRE,...,1,NaN,NaN,True,NaN,01:00:00,GUN,FIREARM,NaN,NaN
78599,2020-02-12,2020-02-02,806 WASHINGTON ST.,WESLACO,HIDALGO,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,NaN,JOSE LUIS,SANDOVAL,JOSE LUIS SANDOVAL,...,1,NaN,NaN,True,NaN,23:21:00,GUN,FIREARM,NaN,NaN
78599,2020-09-03,2020-08-17,1310 N. TEXAS AVE,WESLACO,HIDALGO,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,NaN,MARCO ANTONIO,SIGALA JR.,MARCO ANTONIO SIGALA JR.,...,1,NaN,NaN,True,NaN,15:17:00,GUN,FIREARM,NaN,NaN
78904,2017-02-23,2016-04-16,NaN,VICTORIA,VICTORIA,OTHER - SPECIFY TYPE OF CALL,DECEDENT BROKE INTO THE OFFICERS RESIDENCE.,WADE AUSTIN,KLOESEL,WADE AUSTIN KLOESEL,...,1,NaN,NaN,False,NaN,03:26:00,NaN,NaN,NaN,NaN
88434,NaN,2016-10-24,IH 40 WESTBOUND/MILEPOST 378,NOT IN CITY LIMITS,QUAY (NM),OTHER - SPECIFY TYPE OF CALL,THE INCIDENT ORIGINATED AS A TRAFFIC STOP. THE...,MARIO,MONTES,MARIO MONTES,...,1,NaN,NaN,True,NaN,06:05:00,HANDGUN,FIREARM,NaN,NaN
nan,NaN,2015-12-02,TX 16; .4 MILE N. OF ATASCOSA CO.,NaN,BEXAR,TRAFFIC STOP,NaN,ALLEN,PACHECO,ALLEN PACHECO,...,1,NaN,NaN,True,NaN,NaN,NaN,(DETAILS MISSING),NaN,NaN
nan,NaN,2016-02-27,US HIGHWAY 181 AT LOOP 1604,ELEMENDORF,BEXAR,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,PURSUIT IN STOLEN VEHICLE,FERNANDO,HERNANDEZ,FERNANDO HERNANDEZ,...,1,NaN,NaN,True,NaN,NaN,FIREARM,FIREARM,NaN,NaN
nan,2017-04-13,2017-03-29,FM 1752-NORTH OF FM 898,UNINCORPORATED,FANNIN,TRAFFIC STOP,OTHER - SPECIFY TYPE OF CALL,DENNIS,WASSON II,DENNIS WASSON II,...,1,NaN,NaN,True,NaN,23:11:00,VEHICLE,VEHICLE,NaN,NaN
